In [4]:
using JuMP
using MosekTools
using DynamicPolynomials
using MultivariatePolynomials
using TSSOS
using LinearAlgebra, Random, Plots, Distributions, IterTools, Combinatorics, CSV, Statistics, MLDatasets, DataFrames, Revise, Clustering, Distances
using GaussianMixtures
includet("Functions_Mixtures.jl")
includet("SeparationExperiment.jl")

### Locally used functions

In [5]:
using GaussianMixtures 
using DataFrames
using DataFrames: groupby
using StatsBase       
using StatsPlots 

function random_means()
  
  idx = sample(1:size(X,1), k; replace=false)
  return X[idx, :]
end

function run_em(μ0; X, nIter=100, tol=1e-6, varfloor=1e-3)
    
    μ0 = Matrix(μ0)
    k, d = size(μ0)

   
    σ2 = vec(var(X; dims=1))              # length-d Vector
    Σ0 = repeat(σ2', k, 1)                # k×d Matrix

    # 3) build GMM (skip k‑means by nInit=0)
    g = GMM(k, X; kind=:diag, nInit=10)

    # 4) overwrite init
    g.μ .= μ0
    g.Σ .= Σ0
    g.w .= fill(1/k, k)

    # 5) run EM up to nIter, with varfloor control
    logl = em!(g, X; nIter=nIter, varfloor=varfloor)

    # 6) find when |Δℓ|<tol
    idx = findfirst(i -> abs(logl[i] - logl[i-1]) < tol, 2:length(logl))
    iters = idx === nothing ? length(logl) : idx

    return (
      iterations = iters,
      final_ll   = logl[end],
      logl       = logl,
      model      = g,
    )
end

run_em (generic function with 1 method)

In [6]:
function find_nearest_indices(X::Matrix{Float64}, centers::Matrix{Float64})
    indices = Int[]
    for c in eachrow(centers')
        dists = [sum((X[i, :] .- c).^2) for i in 1:size(X, 1)]
        push!(indices, argmin(dists))
    end
    return indices
end

function adjusted_rand_index(labels_true::Vector{Int}, labels_pred::Vector{Int})
    @assert length(labels_true) == length(labels_pred)

    n = length(labels_true)
    unique_true = sort(unique(labels_true))
    unique_pred = sort(unique(labels_pred))
    n_true = length(unique_true)
    n_pred = length(unique_pred)

    # Confusion matrix: n_ij
    contingency = zeros(Int, n_true, n_pred)
    for (l_true, l_pred) in zip(labels_true, labels_pred)
        i = findfirst(isequal(l_true), unique_true)
        j = findfirst(isequal(l_pred), unique_pred)
        contingency[i, j] += 1
    end

    a = sum(contingency, dims=2)  # true cluster sizes
    b = sum(contingency, dims=1)  # predicted cluster sizes

    comb2(x) = x < 2 ? 0 : x * (x - 1) ÷ 2

    index = sum(comb2(nij) for nij in contingency)

    sum_ai = sum(comb2(ai) for ai in a)
    sum_bj = sum(comb2(bj) for bj in b)

    expected_index = sum_ai * sum_bj / comb2(n)
    max_index = (sum_ai + sum_bj) / 2

    return (index - expected_index) / (max_index - expected_index)
end
function align_labels(true_labels::Vector{Int}, pred_labels::Vector{Int})
    classes = sort(unique(true_labels))
    perms = collect(permutations(classes))

    best_score = -1
    best_aligned = similar(pred_labels)

    for p in perms
        mapping = Dict(classes[i] => p[i] for i in eachindex(classes))
        aligned = [mapping[l] for l in pred_labels]
        score = sum(aligned .== true_labels)

        if score > best_score
            best_score = score
            best_aligned .= aligned
        end
    end

    return best_aligned
end

align_labels (generic function with 1 method)

## Main part

In [7]:
using MLDatasets

mnist = MNIST(split = :train)  
# 1) find the indices of 0 and 1
inds01 = findall(lbl -> lbl in (0,1), mnist.targets)

# 2) subset the images and labels
imgs01  = mnist.features[:, :, inds01]   # 28×28×N01 array
labels01 = mnist.targets[inds01]          # N01‐vector of 0’s and 1’s

inds0 = findall(labels01 .== 0)
imgs0 = imgs01[:, :, inds0]    # now 28×28×N0
X0 = reshape(imgs0, 28*28, :)'   
inds1 = findall(labels01 .== 1)
imgs1 = imgs01[:, :, inds1]    # now 28×28×N0
X1 = reshape(imgs1, 28*28, :)';

In [11]:
# 1) materialize as normal matrices
X0_mat = Matrix(X0)   # now 5923×784 Matrix{Float32}
X1_mat = Matrix(X1)   # now 6742×784 Matrix{Float32}

# 2) stack into one big dataset of 12665×784
X_all = vcat(X0_mat, X1_mat)   # size(X_all) == (5923+6742, 784)

# 3) make the labels (0 for the first block, 1 for the second)
y_all = vcat( fill(0, size(X0_mat,1)),
              fill(1, size(X1_mat,1)) );  # length == 12665

### PCA + relaxations for {0,1}  and {0,1,2}

In [14]:
using MLDatasets, LinearAlgebra, Plots

# Load and prepare data
mnist = MNIST(split=:train)
inds01 = findall(lbl -> lbl in (0, 1), mnist.targets)
imgs01 = mnist.features[:, :, inds01]
labels01 = mnist.targets[inds01]

# Extract images for 0 and 1
imgs0 = imgs01[:, :, labels01 .== 0]
imgs1 = imgs01[:, :, labels01 .== 1]
X0 = reshape(imgs0, 28*28, :)'
X1 = reshape(imgs1, 28*28, :)'

# Combine data and labels
X_all = vcat(Matrix(X0), Matrix(X1))
y_all = vcat(zeros(size(X0, 1)), ones(size(X1, 1)))

# Center the data
X_centered = X_all .- mean(X_all, dims=1)

# Perform SVD
U, S, _ = svd(X_centered)

# Project onto first two PCs
pc1 = U[:, 1] .* S[1]
pc2 = U[:, 2] .* S[2];

In [16]:
# 1) stack into an N×2 matrix
X = hcat(pc1, pc2)    # size(X) == (N, 2)

# 2) scale each column into [–1,1]
X_scaled = hcat( scale_to_minus1_1.(eachcol(X))... );
# size(X_scaled) == (N, 2)
samples=X_scaled;

In [ ]:
# Plot with label
scatter(X_scaled[:,1][y_all .== 0], X_scaled[:,2][y_all .== 0], 
        color=:blue, alpha=0.5, marker=:dot, label="0",
        xlabel="PC1", ylabel="PC2", title="PCA of MNIST Digits 0 and 1")
scatter!(X_scaled[:,1][y_all .== 1], X_scaled[:,2][y_all .== 1], 
        color=:red, alpha=0.5, marker=:dot, label="1")

In [ ]:
using DataFrames, Statistics

# 1) Build a DataFrame from your PC coords + labels
df2 = DataFrame(
  PC1   = X_scaled[:,1],
  PC2   = X_scaled[:,2],
  label = y_all
)

# 2) Group by the label and compute mean & var
stats = combine(
  groupby(df2, :label),
  :PC1 => mean => :mean_PC1,
  :PC1 => var  => :var_PC1,
  :PC2 => mean => :mean_PC2,
  :PC2 => var  => :var_PC2,
)

In [ ]:
n=2
@polyvar m[1:n]
@polyvar sigma[1:n]
#Sm=[(m[1]+1)*(1-m[1]), (m[2]+1)*(1-m[2]),(m[3]+1)*(1-m[3]),(m[4]+1)*(1-m[4])]
Sm=[(m[1])*(1-m[1]), (m[2])*(1-m[2])]
Ssig=[(sigma[1]-0.00005)*(0.15-sigma[1]), (sigma[2]-0.00005)*(0.15-sigma[2])]
S=[vcat(Sm)...,vcat(Ssig)...]
S_normalized=[S[i]/maximum(abs.(coefficients(S[i]))) for i=1:length(S)]

In [ ]:
tr_reg=true        
res=[]
max_order=4
eps_eq=1e-4
eps_tr=1e-2
for i=1:max_order
    println(" d = ", i)
    push!(res, slack_create_SOS_model(n, i, m, sigma, S_normalized, samples, eps_eq, eps_tr; tr_reg))
end

In [ ]:
d=4
RES = analyse_relaxations(res, d, n);

In [ ]:
Curto_F=extract_CF(RES[1], RES[end], binomial(n+d-1,d-1), n, 3)
sort(Curto_F, by = p -> p[1])

In [ ]:
tr_reg=true
eps_tr=1e-1
max_order=4

resTV=[]
for i=1:max_order
    println(" d = ", i)
    push!(resTV, TV_SOS_model(n, i, m, sigma, S_normalized, samples; tr_reg, eps_tr))
end

In [ ]:
d=4
RESTV = analyse_relaxations_TV(resTV, d, n);

In [ ]:
Curto_FTV=extract_CF(RESTV[1], RESTV[end], binomial(n+d-1,d-1),n,3)
sort(Curto_FTV, by = p -> p[1])

In [ ]:
using Clustering
using Statistics
using Random
using Plots

X = samples'  # shape: n×d, here 150×4
C_init = Matrix(hcat(Curto_F...)') 
C_initTV = Matrix(hcat(Curto_FTV...)')  


function find_nearest_indices(X, centers)
    indices = Int[]
    for c in eachrow(centers)
        dists = [sum((X[i, :] .- c).^2) for i in 1:size(X, 1)]
        push!(indices, argmin(dists))
    end
    return indices
end

init_indices = find_nearest_indices(samples, C_init)

init_indicesTV = find_nearest_indices(samples, C_initTV)


result_custom = kmeans(X, 2; init=init_indices)
cost_custom = result_custom.totalcost
iters_custom = result_custom.iterations

result_customTV = kmeans(X, 2; init=init_indicesTV)
cost_customTV = result_customTV.totalcost
iters_customTV = result_customTV.iterations

println("Curto-Fialkow - W2 distance → Cost: ", cost_custom, ", Iterations: ", iters_custom)
println("Curto-Fialkow - TV distance → Cost: ", cost_customTV, ", Iterations: ", iters_customTV)


N = 100
random_costs = Float64[]
random_iters = Int[]

for _ in 1:N
    res = kmeans(X, 2; init=:kmpp)
    push!(random_costs, res.totalcost)
    push!(random_iters, res.iterations)
end


println("Curto-Fialkow - W2 distance → Cost: ", cost_custom, ", Iterations: ", iters_custom)
println("Curto-Fialkow - TV distance → Cost: ", cost_customTV, ", Iterations: ", iters_customTV)
# ---- Statistics ----
mean_cost = mean(random_costs)
std_cost = std(random_costs)
mean_iters = mean(random_iters)
std_iters = std(random_iters)

println("Random → Mean cost: $(round(mean_cost, digits=2)) ± $(round(std_cost, digits=2))")
println("Random → Mean iters: $(round(mean_iters, digits=2)) ± $(round(std_iters, digits=2))")

histogram(random_costs;
    bins=20,
    xlabel="Total cost",
    ylabel="Frequency",
    alpha=0.6,
    label="Random",
    title="KMeans Cost Comparison",
    legend=:topright)
vline!([cost_custom], label="Curto-Fialkow", lw=2, lc=:red)

plot()
histogram(random_iters;
    bins=16,
    xlabel="Iterations",D
    ylabel="Frequency",
    alpha=0.6,
    label="Random",
    #title="KMeans IteSSration Comparison",
    legend=:topright)
vline!([iters_custom], label="Curto-Fialkow-W2", lw=4, lc=:red)
vline!([iters_customTV], label="Curto-Fialkow-TV", lw=4, lc=:orange)

In [ ]:
using Plots, PGFPlotsX, LaTeXStrings, Colors

# fixed colors
col_w2  = RGB(0.0,    0.6056, 0.9787)  # blue:  CF–W2
col_tv  = RGB(0.8889, 0.4356, 0.2781)  # orange: CF–TV
col_rnd = RGB(0.2422, 0.6433, 0.3044)  # green:  Random

# --- Preview in notebook (GR) ---
gr()
default(size=(560,360), framestyle=:box, grid=false, legend=:topright)

plt_iter = histogram(random_iters; bins=16,
    label="Random ± 1 std",
    fillcolor=col_rnd, linecolor=col_rnd, alpha=0.45,   # <- force green
    xlabel="Iterations", ylabel="Frequency")

vline!(plt_iter, [iters_custom];    label="Curto–Fialkow–W2", lw=2, color=col_w2)
vline!(plt_iter, [iters_customTV];  label="Curto–Fialkow–TV", lw=2, color=col_tv)
display(plt_iter)




In [ ]:
# --- Export same look to TikZ ---
pgfplotsx()
plt_iter_tex = histogram(random_iters; bins=16,
    label=L"\text{Random} \pm \text{ 1 std}",
    fillcolor=col_rnd, linecolor=col_rnd, alpha=0.45,
    xlabel=L"\text{Iterations}", ylabel=L"\text{Frequency}")

vline!(plt_iter_tex, [iters_custom];   label=L"\text{Curto–Fialkow–W2}", lw=2, color=col_w2)
vline!(plt_iter_tex, [iters_customTV]; label=L"\text{Curto–Fialkow–TV}", lw=2, color=col_tv)

#savefig(plt_iter_tex, "kmeans_iter_hist.tikz")

In [ ]:
using Clustering
using Statistics
using Random
using Plots

X = samples'  # shape: n×d, here 150×4
C_init = Matrix(hcat(Curto_F...)') 
C_initTV = Matrix(hcat(Curto_FTV...)')  


function find_nearest_indices(X, centers)
    indices = Int[]
    for c in eachrow(centers)
        dists = [sum((X[i, :] .- c).^2) for i in 1:size(X, 1)]
        push!(indices, argmin(dists))
    end
    return indices
end

init_indices = find_nearest_indices(samples, C_init)

init_indicesTV = find_nearest_indices(samples, C_initTV)


result_custom = kmeans(X, 3; init=init_indices)
cost_custom = result_custom.totalcost
iters_custom = result_custom.iterations

result_customTV = kmeans(X, 3; init=init_indicesTV)
cost_customTV = result_customTV.totalcost
iters_customTV = result_customTV.iterations

println("Curto-Fialkow - W2 distance → Cost: ", cost_custom, ", Iterations: ", iters_custom)
println("Curto-Fialkow - TV distance → Cost: ", cost_customTV, ", Iterations: ", iters_customTV)


N = 100
random_costs = Float64[]
random_iters = Int[]

for _ in 1:N
    res = kmeans(X, 3; init=:kmpp)
    push!(random_costs, res.totalcost)
    push!(random_iters, res.iterations)
end


println("Curto-Fialkow - W2 distance → Cost: ", cost_custom, ", Iterations: ", iters_custom)
println("Curto-Fialkow - TV distance → Cost: ", cost_customTV, ", Iterations: ", iters_customTV)
# ---- Statistics ----
mean_cost = mean(random_costs)
std_cost = std(random_costs)
mean_iters = mean(random_iters)
std_iters = std(random_iters)

println("Random → Mean cost: $(round(mean_cost, digits=2)) ± $(round(std_cost, digits=2))")
println("Random → Mean iters: $(round(mean_iters, digits=2)) ± $(round(std_iters, digits=2))")

histogram(random_costs;
    bins=20,
    xlabel="Total cost",
    ylabel="Frequency",
    alpha=0.6,
    label="Random",
    title="KMeans Cost Comparison",
    legend=:topright)
vline!([cost_custom], label="Curto-Fialkow", lw=2, lc=:red)

plot()
histogram(random_iters;
    bins=16,
    xlabel="Iterations",
    ylabel="Frequency",
    alpha=0.6,
    label="Random",
    #title="KMeans IteSSration Comparison",
    legend=:topright)
vline!([iters_custom], label="Curto-Fialkow-W2", lw=4, lc=:red)
vline!([iters_customTV], label="Curto-Fialkow-TV", lw=4, lc=:orange)

In [ ]:
using Plots, PGFPlotsX, LaTeXStrings, Colors

# fixed colors
col_w2  = RGB(0.0,    0.6056, 0.9787)  # blue:  CF–W2
col_tv  = RGB(0.8889, 0.4356, 0.2781)  # orange: CF–TV
col_rnd = RGB(0.2422, 0.6433, 0.3044)  # green:  Random

# --- Preview in notebook (GR) ---
gr()
default(size=(560,360), framestyle=:box, grid=false, legend=:topright)

plt_iter = histogram(random_iters; bins=16,
    label="Random ± 1 std",
    fillcolor=col_rnd, linecolor=col_rnd, alpha=0.45,   # <- force green
    xlabel="Iterations", ylabel="Frequency")

vline!(plt_iter, [iters_custom];    label="Curto–Fialkow–W2", lw=2, color=col_w2)
vline!(plt_iter, [iters_customTV];  label="Curto–Fialkow–TV", lw=2, color=col_tv)
display(plt_iter)

In [ ]:
# --- Export same look to TikZ ---
pgfplotsx()
plt_iter_tex = histogram(random_iters; bins=16,
    label=L"\text{Random} \pm \text{ 1 std}",
    fillcolor=col_rnd, linecolor=col_rnd, alpha=0.45,
    xlabel=L"\text{Iterations}", ylabel=L"\text{Frequency}")

vline!(plt_iter_tex, [iters_custom];   label=L"\text{Curto–Fialkow–W2}", lw=2, color=col_w2)
vline!(plt_iter_tex, [iters_customTV]; label=L"\text{Curto–Fialkow–TV}", lw=2, color=col_tv)

#savefig(plt_iter_tex, "kmeans_iter_hist_012_2comp.tikz")